# Исследование объявлений о продаже квартир

Для анализа имеется архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Основная задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

### Откроем файл с данными и изучим общую информацию. 

In [1]:
#Импортируем библиотеки
import pandas as pd

In [2]:
#Откроем файл real_estate_data.csv
try:
    data = pd.read_csv('Проект: Исследовательский анализ данных/real_estate_data.csv', sep='\t')
except:
    data = pd.read_csv('real_estate_data.csv', sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: 'real_estate_data.csv'

In [ ]:
#Выведем первые 5 строк анализируемой таблицы
pd.set_option('display.max_columns', None)
data.head()

In [ ]:
#Рассмотрм основную информацию
data.info()

На данный момент в таблице 23699 строк. 
По полученной информации можно сказать

- is_apartment нужно преобразовать в тип bool, так как по нему отражена информация яляется ли недвижимость апартаментами
- floors_total нужно преобразовать в int64
- balcony нужно преобразовать в int64
- parks_around3000 нужно преобразовать в тип int64
- ponds_around3000 нужно преобразовать в тип int64
- days_exposition нужно преобразовать в тип int64, так как количество не может быть вещественным типом данных

In [ ]:
#Рассмотрим графы таблицы
data.columns

Расшифровка наименований граф

0)   total_images -         всего изображений (фото)

1)   last_price -           последняя цена

2)  total_area -           общая площадь

3)   first_day_exposition - день выставления на продажу

4)   rooms -                количество комнат

5)   ceiling_height -       высота потолка

6)   floors_total -         количество этажей

7)   living_area -          жилая площадь 

8)   floor -                этаж

9)  is_apartment -         апартаменты

10)  studio -               студия  

11)  open_plan -            свободная планировка

12)  kitchen_area -         площадь кухни

13)  balcony -              число балконов

14)  locality_name -        где находится 

15)  airports_nearest -     ближайший аэропорт (м)

16)  cityCenters_nearest -  расстояние до центра (м)

17)  parks_around3000 -     количество парков рядом (в пределах 3000м)

18)  parks_nearest -        расстояние до парка (м)

19)  ponds_around3000 -    количество озер/прудов рядом (в пределах 3000м)

20)  ponds_nearest -        расстояние до озера/пруда (м)

21)  days_exposition -      сколько дней висит объявление


In [ ]:
#Построим гистограммы для всех столбцов
data.hist(figsize=(15, 20));

### Предобработка данных

#### Общие сведения

In [ ]:
#Переименуем графы в соответствии с правилами
data = data.rename(columns={'cityCenters_nearest': 'city_centers_nearest'}) 

In [ ]:
#Рассмотрим общее количество пропусков по таблице в целом
data.isna().sum()

#### Удаление дубликатов

In [ ]:
#Проверим таблицу на наличие дубликатов на начальном этапе
data.duplicated().sum()

Рассмотрим значения граф на наличие аномалий и дубликатов, ниже представлены данные по графам, где были обнаружны аномалии или дубликаты.

In [ ]:
#Рассмотрим уникальные значения графы ceiling_height
data['ceiling_height'].unique()

В значениях присутствуют неправдоподобные данные, такие как 25.0, 32.0, 27.0, 24.0, 26.0, 14.0, 20.0, 22.6, 27.5, 100.0 метров.

In [ ]:
#Заменим аномалии на предполагаемы значения
data = data.replace({'ceiling_height':
                     {25.:2.5, 32.:3.2, 27.:2.7, 
                      24.:2.4, 26.:2.6, 14.:1.4, 
                      20.:2.0, 22.6:2.26, 27.5:2.75, 
                      100.:10.0}})

In [ ]:
#Проверим данные после замены
data['ceiling_height'].unique()

In [ ]:
#Рассмотрим значения столбца locality_name
data['locality_name'].unique()

При рассмотрении значений были выявлены неявные дубликаты, такие как поселок/посёлок и другие.

In [ ]:
#Рассмотрим изначальное количество значений по каждому населенному пункту
data['locality_name'].value_counts().head(10)

In [ ]:
#Проведем замену найденных дубликатов
data['locality_name'] = (
    data['locality_name'].str.replace('поселок', 'посёлок')
)
data['locality_name'] = (
    data['locality_name'].str.replace('городской посёлок', 
                                      'посёлок городского типа')
)
data['locality_name'] = (
    data['locality_name'].str.replace('садоводческое некоммерческое товарищество', 
                                      'садовое товарищество')
)
data['locality_name'] = (
    data['locality_name'].str.replace('посёлок при железнодорожной станции', 
                                      'посёлок станции')
)
data['locality_name'] = (
    data['locality_name'].str.replace('посёлок при железнодорожной станции',
                                      'посёлок')
)
data['locality_name'] = (
    data['locality_name'].str.replace('посёлок станции', 'посёлок')
)
data['locality_name'] = (
    data['locality_name'].str.replace('коттеджный посёлок', 'посёлок')
)

In [ ]:
#Проведем замену найденных дубликатов в процессе анализа пропущенных значенй
data['locality_name'] = (
    data['locality_name'].str.replace('посёлок Мурино', 'Мурино')
)
data['locality_name'] = (
    data['locality_name'].str.replace('село Никольское', 'Никольское')
)
data['locality_name'] = (
    data['locality_name'].str.replace('деревня Рабитицы', 'посёлок Рабитицы')
)

In [ ]:
#Выведем результат
data['locality_name'].unique()

In [ ]:
#Проверим количесто значений по каждому населенному пункту после проведенных изменений
data['locality_name'].value_counts().head(10)

Количество населенных пунктов увеличилось, за счет переименовывания дубликатов.
Например:
1. до посёлок Мурино         522
2. после Мурино                 590

Проанализируем, возможно ли избавиться от пропусков: необходимо на их место прописать значение или удалить эти строки целиком.

Графы floors_total(86), locality_name(49) содержат пропуски, которые невозможно логически заполнить. Количество данных пропусков составляет менее 1%, поэтому строки без данных по этим графам можно удалить, на общую статистику данное действие не будет иметь существенного влияния.

In [ ]:
#Удалим строки с пропусками в данных по графам floors_total, locality_name и 
#переведем столбец floors_total к целочисленному типу данных 
data = data.dropna(subset=['floors_total', 'locality_name']) 
data['floors_total'] = data['floors_total'].astype('int')
data = data.reset_index(drop=True)

#### Преобразование is_apartment и balcony

Будем считать, что отсутствие значений в графах is_apartment и balcony означает, что данные просто оставили пустыми, так как подразумевалось отсутствие информации. Поэтому на место пустых значений можно поставить 0.

In [ ]:
#Заменим пустые значения в графах is_apartment и balcony на 0,
#также преобразуем is_apartment к типу bool
#balcony к типу int
data['is_apartment'] = data['is_apartment'].fillna(value=0)
data['balcony'] = data['balcony'].fillna(value=0)

data['is_apartment'] = data['is_apartment'].astype('bool')
data['balcony'] = data['balcony'].astype('int')

Мы преобразовали is_apartment к булевому типу данных, так как в данной графе указано или Да, или Нет, то есть является ли квартира апартаментами, других вариаций ответа в графе быть не может.
Также количество балконом не может быть вещественным типом данных.

In [ ]:
#Проверим в каких графах остались еще пропуски
data.isna().sum()

#### Преобразование ceiling_height

В графе ceiling_height имеются пропуски, заполним их медианным значением по этому населенному пункту.

In [ ]:
#Выведем медианные значения по высоте потолков по населенным пунктам 
data_pivot_hight = (
    data
    .pivot_table(index=(data[~data['ceiling_height'].isna()]['locality_name']), values='ceiling_height', aggfunc='median')
    .reset_index()
)
display(data_pivot_hight.head(10))

In [ ]:
#Сопоставим значения по всем строкам
ceiling_height = data.merge(data_pivot_hight, on='locality_name', how='left')

In [ ]:
#Запишем полученные значения в пустые ячейки
data['ceiling_height'] = data['ceiling_height'].fillna(ceiling_height['ceiling_height_y'])

In [ ]:
#Проверим есть ли данные по которым не определена медиана
data[data['ceiling_height'].isna()]['locality_name'].value_counts()

In [ ]:
#Для населенных пунктов, где не была определена 
#медиана из-за отсутствия данных запишем данные по санпину
data['ceiling_height'] = data['ceiling_height'].fillna(2.50)
data[data['ceiling_height'].isna()]

#### Преобразование kitchen_area

In [ ]:
#Выведем медианные значения по площади кухни в зависимлсти от населенного пункта 
data_pivot_kitchen_area = (
    data
    .pivot_table(index=(data[~data['kitchen_area'].isna()]['locality_name']), values=['kitchen_area','total_area'], aggfunc='mean')
    .reset_index()
)
display(data_pivot_kitchen_area)

In [ ]:
#Расситаем соотношение площадей (соотношение площади кухни к общей) по населенному пункту
data_pivot_kitchen_area['relation'] = data_pivot_kitchen_area['kitchen_area'] / data_pivot_kitchen_area['total_area']

In [ ]:
#Сопоставим значения
kitchen_area = data.merge(data_pivot_kitchen_area, on='locality_name', how='left')

In [ ]:
#Запишем полученные значения в пустые ячейки
data['kitchen_area'] = data['kitchen_area'].fillna(kitchen_area['relation'] * data['total_area'])

In [ ]:
#Проверим наличие пропусков
data[data['kitchen_area'].isna()]['locality_name'].value_counts()

In [ ]:
#Для данных населенных пунктов нет среднего, так как нет данных
#присвоим им среднее соотношение кухни к общей площади 
data['kitchen_area'] = data['kitchen_area'].fillna(kitchen_area['relation'].mean())

In [ ]:
#Проверим наличие пропусков
data[data['kitchen_area'].isna()]

#### Преобразование living_area

In [ ]:
#Выведем медианные значения по жилой площади в зависимлсти от населенного пункта 
data_pivot_living_area = (
    data
    .pivot_table(index=(data[~data['living_area'].isna()]['locality_name']), values='living_area', aggfunc='mean')
    .reset_index()
)
display(data_pivot_living_area)

In [ ]:
#проверим данные на логическую корректность, есть ли квартиры, 
#в которых жил.площадь больше разницы площадей других помещений
data[data['living_area'] > (data['total_area'] - data['balcony'] - data['kitchen_area'])]

К жилой площади относятся только те помещения, которые пригодны для проживания: гостиная, спальня, детская. Сюда не относятся: кухня, санузлы, коридор, кладовые и гардеробные. В полезную площадь включают все используемые помещения. Сюда могут войти балкон и терраса с отоплением, которые не вошли в общую площадь.

Возможно в данных была ошибка.

In [ ]:
#Сопоставим значения
living_area = data.merge(data_pivot_living_area, on='locality_name', how='left')

In [ ]:
#Запишем полученные значения в пустые ячейки
data['living_area'] = data['living_area'].fillna(living_area['living_area_y'])

In [ ]:
#Проверим наличие пропусков
data[data['living_area'].isna()]['locality_name'].value_counts()

In [ ]:
#Заполним данные разницей полщадей
data['living_area'] = (
    data['living_area']
    .fillna(data['total_area'] - data['balcony'] - data['kitchen_area'])
)

In [ ]:
#Проверим наличие пропусков
data[data['living_area'].isna()]

In [ ]:
#Проверим наличие пропусков в других графах
data.isna().sum()

#### Преобразование airports_nearest, city_centers_nearest, parks_nearest и parks_around3000, ponds_nearest и ponds_around3000

Картографические данные не могут быть заполнены нами с достаточной точностью, поэтому оставим пропуски в графах.

Парков в радиусе 3000м нет. Значение до ближайшего парка (parks_nearest) мы не можем проставить 0, так как расстояние до парка наоборот больше 3000м; как среднюю также не можем заполнить, потому что имеется большое значение неизвестных картографических параметров (5500), что будет влиять на достоверность данных.

Озер/прудов в радиусе 3000м нет. Значение до ближайшего озера (ponds_nearest) мы не можем проставить 0, так как расстояние до озера наоборот больше 3000м; как среднюю также не можем заполнить, потому что имеется большое значение неизвестных картографических параметров (5500), что будет влиять на достоверность данных.

#### Преобразование days_exposition

Отсутствие значения по графе days_exposition означает активное объявление, квартира не была продана. Поэтому данные не преобразуем.

#### Проведем анализ аномальных значений по цене квартиры

In [ ]:
#Посторим диаграмму размаха по цене квартиры
data.boxplot('last_price');

На диаграмме видны аномальные значения.

In [ ]:
#выведем цены квартир по убыванию
data.sort_values('last_price', ascending=False)['last_price'].head(20)

In [ ]:
#количество записей непреобразованной таблицы
data.shape[0]

In [ ]:
#перезапишем в таблицу значения, цена квартиры которых меньше 200000000
data = data[data['last_price'] < 200000000]

In [ ]:
#количество записей преобразованной таблицы
data.shape[0]

In [ ]:
#процент исключенных записей менее 1%
(23565 - 23556) / 23565

#### Проведем анализ аномальных значений по количеству комнат

In [ ]:
#рассмотрим количество записей для квартир на разных этажах
data['rooms'].value_counts()

In [ ]:
#для квартир выше 7 этажа слишком маленькая выборка, исключим данные значеения
data = data[data['rooms'] < 7]

In [ ]:
#количество записей преобразованной таблицы
data.shape[0]

In [ ]:
#процент исключенных записей менее 1%
(23556 - 23469) / 23556

#### Проведем анализ аномальных значений по общей площади квартиры

In [ ]:
#Посторим диаграмму размаха
data.boxplot('total_area');

In [ ]:
#выведем данные по общим площадям квартир по убыванию
data.sort_values('total_area', ascending=False)['total_area'].head(40)

In [ ]:
#исключитм значения по площадям меньше 350 м2
data = data[data['total_area'] < 350.0]

In [ ]:
#количество записей преобразованной таблицы
data.shape[0]

In [ ]:
#процент исключенных записей менее 1%
(23469 - 23453) / 23469

#### Итоги по результатам предобработки данных

In [ ]:
#Проверим количество значений
data.shape

In [ ]:
#Проверим количество пропусков
data.isna().sum()

In [ ]:
data.info()

После преобразований, количество трок сократилось на 246 строк
- с 23699 строк
- до 23453 строк

По полученной информации можно сказать

- is_apartment преобразован в тип bool

- floors_total преобразован в int64 

- balcony преобразован в тип int64 

- parks_around3000 - не преобразован, так как в столбце все еще присутствуют незаполненные данные, которые не могут быль заполнены числовыми значеними

- ponds_around3000 - не преобразован, так как в столбце все еще присутствуют незаполненные данные, которые не могут быль заполнены числовыми значеними

- days_exposition - не преобразован, так как в столбце все еще присутствуют незаполненные данные, которые не могут быль заполнены числовыми значеними

### Посчитайте и добавьте в таблицу новые столбцы

In [ ]:
#Расситаем стоимость одного квадратного метра и 
#запишем данные в таблицу
data['price_metr'] = data['last_price'] / data['total_area']

In [ ]:
#Запишем в таблицу день недели размещения объявления
#Так как время не указано, выведем только день публикации
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%d')
data['weekday'] = data['first_day_exposition'].dt.weekday

In [ ]:
#Рассмотрим полученны данные
data.head()

Проверим 2019-03-07 - четверг (3) расчет верен

In [ ]:
#Запишем в таблицу месяц первого размещения
data['month'] = data['first_day_exposition'].dt.month

In [ ]:
#Запишем в таблицу год первого размещения
data['year'] = data['first_day_exposition'].dt.year

In [ ]:
#Категоризируем данные по размещению квартиры относительно всех этажей дома
def categorize_floor(row):
    floor = row['floor']
    floors_total = row['floors_total']
    try:
        if floor == floors_total:
            return 'последний'
        elif floor == 1:
            return 'первый'
        elif 1 < floor < floors_total:
            return 'другой'
    except:
        pass

In [ ]:
#Применим функцию к таблице и запишем категории для каждой строки
data['category_floor'] = data.apply(categorize_floor, axis=1)

In [ ]:
#Переведем расстояние до центра из метров в километры
data['city_centers_nearest_km'] = round(data['city_centers_nearest'] / 1000)

In [ ]:
#Проверим данные
data.head()

In [ ]:
#Выведем основную информацию по таблице
data.info()

На основе данных были рассчитаны и добавлены следующие столбцы:

- price_metr - цена за квадратный метр
- weekday - день недели размещения объявления
- month - месяц размещения объявления
- year - год размещения объявления
- category_floor - категории этажей
- city_centers_nearest_km - удаленность от центра в км

### Проведите исследовательский анализ данных

#### Изучение параметров

In [ ]:
#Построекние гистограммы для общей площади
data['total_area'].hist(range=(0, 300));

По полученным данным можно сказать, что площадь квартиры подчинена распределению Пуассона. 
Чаще всего встречаются квартиры в промежутке от 45 до 75 метров квадратных.
При увеличении площади квартиры уменьшается количество предложений, что справедиво для промежутка от 75 м2.

In [ ]:
#Построекние гистограммы для жилой площади
data['living_area'].hist(range=(0, 200));

Жилая площадь квартиры подчинена распределению Пуассона. 
Чаще всего встречаются квартиры с жилой площадью в промежутке от 25 до 40 метров квадратных. При увеличении жилой площади квартиры уменьшается количество предложений, что справедиво для промежутка от 40 м2.

In [ ]:
#Построекние гистограммы для площади кухни
data['kitchen_area'].hist(range=(0, 50));

Площадь кухни, также как и полщади других территорий квартиры, подчинена распределению Пуассона. Чаще всего встречаются квартиры с площадью кухни в промежутке от 8 до 10 метров квадратных. При увеличении площади кухни уменьшается количество предложений, что справедиво для промежутка от 10 м2.

In [ ]:
#Построекние гистограммы по цене
data['last_price'].hist(range=(0, 35000000), bins=100);

Чаще всего в представленных данных встречается цена на квартиры от 3 до 4 млн.р. При дальнейшем увеличении цены предложение резко уменьшается.

In [ ]:
#Построекние гистограммы по количеству комнат
data['rooms'].hist(range=(0, 8), bins=8);

По полученным данным можно сказать, чаще всего встречаются квартиры с 1-3 комнатами. При увеличении количества комнат в квартире уменьшается количество предложений.

In [ ]:
#Построекние гистограммы по высоте потолков
data['ceiling_height'].hist(range=(2, 4.5));

Чаще всего в предложениях встречаются квартиры с высотой потолка от 2,5 до 2,75 метров. Что соответствует данным санпина.

In [ ]:
#Построекние гистограммы по этажу квартиры
data['floor'].hist(range=(0, 30), bins=30);

По данным можно сделать вывод, что чаще всего продают квартиры первых этажей. Также по графику видно резкое снижение предложений на квартиры, находящиеся выше 5 этажа. Данный факт может быть результатом большого количества небольших городов, представленных в выборке, в которых не часто распространены здания выше 5 этажей.

In [ ]:
#Построекние гистограммы по категории этажей
data['category_floor'].hist(bins=3);

По данным графика можно отметить, что чаще всего продаются квартиры находящиеся не на первом, и не на последнем этажах. Возможными причинами могут быть бытовые факторы.

In [ ]:
#Построекние гистограммы по количеству этаженй дома
data['floors_total'].hist(range=(0, 40));

В выборке чаще всего встречались здания по 5 этажей и меньше всего зданий высотой от 30 этажей, которые присущи большим городам.

In [ ]:
#Построекние гистограммы по удаленности от центра
data['city_centers_nearest'].hist();

Большую часть предложений составляют квартиры, которые отдалены от центра более чем на 10-20 км, по мере удаления предложение снижается.

In [ ]:
#Построекние гистограммы по расстоянию до аэропорта
data['airports_nearest'].hist();

Чаще всего продаются квартиры отдаленные от аэропорта на расстояние 20-25км.

In [ ]:
#Построекние гистограммы по расстоянию до парка
data['parks_nearest'].hist(range=(0, 3000), bins=20);

Для представленных в выборке квартир распространена характеристика - отдаленность от парка 400-600 метров.

In [ ]:
#Построекние гистограммы по дням недели регистрации объявления
data['weekday'].hist(bins=7);

Квартиры чаще всего выставляются на продажу в течение рабочей недели. Причиной этого может быть большой процент таких  пользователей, как агенства недвижимости.

In [ ]:
#Построекние гистограммы по месяцам регистрации объявления
data['month'].hist(bins=12);

Большая активность пользователей наблюдается в периоды с февраля по май и с июня по ноябрь, в месяцах: май, декабрь и январь отмечается спад активности, возможной причиной могут быть праздничные дни.

#### Анализ графы days_exposition

In [ ]:
#Построекние гистограммы по количеству дней активности объявления
data['days_exposition'].hist(range=(0, 1750), bins=50);

In [ ]:
#Построекние гистограммы по количеству дней активности объявления
data['days_exposition'].hist(range=(0, 100), bins=100);

На графике за 100 дней ярко выражены пики активности, по приблизительным подсчетам, частота повторений  данных периодов активности равна 1-2 неделям.

In [ ]:
#Выведем описание для данной графы
data['days_exposition'].describe()

Как мы можем заметить: 
- среднее количество дней составило 180.380685
- медианное количество дней 95.000000

Можно сделать вывод, что для анализируемой выборки характерно нестабильное время активности объявления. Наблюдается большое среднеквадратическое отклонение, равное 219.328653.
Причиной может служить большой разброс по населенным пунктам, с разной степенью удаленности от административных центров и с разным количеством жителей.

- Самая быстрая сделака состоялась через 1 день.
- Квартира находившаяся на продаже дольше всего не покупалась 1580 дней. 

#### Анализ зависимости общей стоимости объекта от разных факторов

##### Анализ зависимости общей стоимости объекта от общей площади квартиры

In [ ]:
#отношение общей стоимости к общей площади квартиры
data_pivot_total_area = (
    data
    .pivot_table(index=(data['total_area']), values='last_price', aggfunc='median')
    .reset_index()
)
display(data_pivot_total_area.sort_values('total_area', ascending=False))

In [ ]:
#построим диаграмму рассеяния
data_pivot_total_area.plot(x='total_area', y='last_price', kind='scatter', grid=True, alpha=0.3);

In [ ]:
#рассчитаем коэффициент корреляции
data_pivot_total_area['last_price'].corr(data_pivot_total_area['total_area'])

Согласно полученным данным наблюдается прямая зависимость между общей стоимостью и общей площадью квартиры. Об этом свидетельствует положение точек на графике, которые в свою очередь образуют размытую восходящую линию. Также коэффициент корреляции (74%) свидетельствует о достаточно сильной линейной зависимости. 

##### Анализ зависимости общей стоимости объекта от жилой площади квартиры

In [ ]:
#отношение общей стоимости к жилой площади квартиры
data_pivot_living_area = (
    data
    .pivot_table(index=(data['living_area']), values='last_price', aggfunc='median')
    .reset_index()
)
display(data_pivot_living_area.sort_values('living_area', ascending=False))

In [ ]:
#построим диаграмму рассеяния
data_pivot_living_area.plot(x='living_area', y='last_price', kind='scatter', grid=True, alpha=0.3);

In [ ]:
#рассчитаем коэффициент корреляции
data_pivot_living_area['last_price'].corr(data_pivot_living_area['living_area'])

Согласно полученным данным наблюдается умеренная зависимость между общей стоимостью и жилой площадью квартиры. Точки на диаграмме рассеяны в большей степени, чем в предыдущей зависимости.
Коэффициент корреляции (69%) свидетельствует о умеренной линейной зависимости. Данное соотношение говорит о том, что имеются сторонние факторы влияющие на переменные.

##### Анализ зависимости общей стоимости объекта от площади кухни

In [ ]:
#отношение общей стоимости к площади кухни квартиры
data_pivot_kitchen_area = (
    data
    .pivot_table(index=(data['kitchen_area']), values='last_price', aggfunc='median')
    .reset_index()
)
display(data_pivot_kitchen_area.sort_values('kitchen_area', ascending=False))

In [ ]:
#построим диаграмму рассеяния
data_pivot_kitchen_area.plot(x='kitchen_area', y='last_price', kind='scatter', grid=True, alpha=0.3);

In [ ]:
#рассчитаем коэффициент корреляции
data_pivot_kitchen_area['last_price'].corr(data_pivot_kitchen_area['kitchen_area'])

Данные взаимосвязи площади кухни и общей стоимости квартиры аналогичны отношению жилой площади и общей стоимости квартиры. На диаграмме наблюдается большая степень рассеивания данных с увеличением площади кухни. Коэффициент корреляции (70%) свидетельствует о сильной линейной зависимости.

##### Анализ зависимости общей стоимости объекта от количества комнат квартиры

In [ ]:
#отношение общей стоимости к количеству комнат квартиры
data_pivot_rooms = (
    data
    .pivot_table(index=(data['rooms']), values='last_price', aggfunc='median')
    .reset_index()
)
display(data_pivot_rooms.sort_values('rooms', ascending=False))

In [ ]:
#построим диаграмму
data_pivot_rooms.plot(x='rooms', y='last_price', grid=True, alpha=0.7);

In [ ]:
#рассчитаем коэффициент корреляции
data_pivot_rooms['last_price'].corr(data_pivot_rooms['rooms'])

Данные взаимосвязи количества комнат и общей стоимости на диаграмме представлены плавной возрастающей. Коэффициент корреляции (95%) свидетельствует о практически абсолютной линейной зависимости цены квартиры от количества комнат.

##### Анализ зависимости общей стоимости объекта от этажа

In [ ]:
#отношение общей стоимости к этажу квартиры
data_pivot_category_floor = (
    data
    .pivot_table(index=(data['category_floor']), values='last_price', aggfunc='median')
    .reset_index()
)
display(data_pivot_category_floor.sort_values('category_floor', ascending=False))

In [ ]:
#построим диаграмму
data_pivot_category_floor.plot(x='category_floor', y='last_price', grid=True, alpha=0.7);

По полученным данным можно сделать вывод, что первые и последние этажи чаще всего имеют стоимоть ниже, чем квартиры на других этажах. Возможными причинами могут быть бытовые особенности.

##### Анализ зависимости общей стоимости объекта от дня недели размещения объявления

In [ ]:
#отношение общей стоимости к дню недели размещения объявления
data_pivot_weekday = (
    data
    .pivot_table(index=(data['weekday']), values='last_price', aggfunc='median')
    .reset_index()
)
display(data_pivot_weekday.sort_values('weekday', ascending=False))

In [ ]:
#построим диаграмму
data_pivot_weekday.plot(x='weekday', y='last_price', grid=True, alpha=0.7);

Согласно диаграмме стоимость квартиры выставлялась выше в рабочие дни, для выходных, стоимость квартир указывалась ниже. Возможной причиной может быть использование ресурса агентами по недвижимости.

##### Анализ зависимости общей стоимости объекта от месяца размещения объявления

In [ ]:
#отношение общей стоимости к месяцу размещения объявления
data_pivot_month = (
    data
    .pivot_table(index=(data['month']), values='last_price', aggfunc='median')
    .reset_index()
)
display(data_pivot_month.sort_values('month', ascending=False))

In [ ]:
#построим диаграмму
data_pivot_month.plot(x='month', y='last_price', grid=True, alpha=0.7);

По полученным данным сделаем вывод, что общая стоимость квартиры слабо зависит от месяца регистрации объявления. Стоимость в данном случае зависит от сторонних факторов. 

##### Анализ зависимости общей стоимости объекта от года размещения объявления

In [ ]:
#отношение общей стоимости к году размещения объявления
data_pivot_year = (
    data
    .pivot_table(index=(data['year']), values='last_price', aggfunc='median')
    .reset_index()
)
display(data_pivot_year.sort_values('year', ascending=False))

In [ ]:
#построим диаграмму
data_pivot_year.plot(x='year', y='last_price', grid=True, alpha=0.7);

Согласно графику, стоимость квартир с 2014 года по 2017 падала, в 2017 году замечено восходящее движение графика. Возможной причиной могут быть сторонние факторы, такие как инфляция и ситуация в стране.

#### Анализ 10 населенных пунктов с наибольшим количеством объявлений

In [ ]:
#Выведем 10 населенных пунктов с наибольшим количеством объявлений
data['locality_name'].value_counts().head(10)

In [ ]:
#Сохраним данные в DataFrame
good_data = data.loc[data['locality_name'].isin(['Санкт-Петербург', 'Мурино', 'посёлок Шушары', 'Всеволожск', 
                                'Пушкин', 'Колпино', 'посёлок Парголово', 'Гатчина', 'деревня Кудрово', 'Выборг'])]

In [ ]:
#Проверим полученную таблицу
good_data.head()

In [ ]:
#Создадим выборку данных, отсортируем данные по уменьшению цены за кв.метр
data_pivot_price_metr = (
    good_data
    .pivot_table(index=(good_data['locality_name']), values='price_metr', aggfunc='median')
    .reset_index()
)
display(data_pivot_price_metr.sort_values('price_metr', ascending=False))

По полученным данным, самым дорогим по стоимости квадратного метра стал Санкт-Петербург, медианная стоимость составила  104761.90р.
Самым дешевым из представленных стал квадратный метр в Выборге, его стоимость составила 58158.32р.

#### Анализ зависимости расстояния от центра на стоимость квартиры

In [ ]:
#Создадим DF для данных по Санкт-Петербургу
data_spb = data.loc[data['locality_name'] == 'Санкт-Петербург'].copy()
data_spb['center_km_price'] = ""

In [ ]:
#Рассчитаем цену километра до центра
def def_center_km_price(row):
    city_centers_nearest_km = row['city_centers_nearest_km']
    center_km_price = row['center_km_price']
    last_price = row['last_price']
    try:
        center_km_price = last_price / city_centers_nearest_km
    except ZeroDivisionError:
        center_km_price = last_price
    return center_km_price

In [ ]:
#Применим функцию к таблице и запишем категории для каждой строки
data_spb['center_km_price'] = data_spb.apply(def_center_km_price, axis=1)

In [ ]:
#Выведем медианные значения по расстоянию до центра 
data_pivot_spb_median = (
    data_spb
    .pivot_table(index=['city_centers_nearest_km'], values='center_km_price', aggfunc='median')
    .reset_index()
)

display(data_pivot_spb_median)

In [ ]:
#Построим диаграмму рассеяния
data_pivot_spb_median.plot(x='city_centers_nearest_km', 
                           y='center_km_price',  
                           grid=True, 
                           alpha=0.7, 
                           figsize=(20, 10),
                           marker='o');

In [ ]:
#Рассчитаем коэффициент корреляции
data_pivot_spb_median['city_centers_nearest_km'].corr(data_pivot_spb_median['center_km_price'])

In [ ]:
#Посторим диаграмму размаха
data_pivot_spb_median.boxplot('center_km_price');

In [ ]:
#Выведем средние значения по расстоянию до центра 
data_pivot_spb_mean = (
    data_spb
    .pivot_table(index=['city_centers_nearest_km'], values='center_km_price', aggfunc='mean')
    .reset_index()
)
display(data_pivot_spb_mean)

In [ ]:
#Построим диаграмму рассеяния
data_pivot_spb_mean.plot(x='city_centers_nearest_km', 
                         y='center_km_price', 
                         grid=True, 
                         alpha=0.7, 
                         figsize=(20, 10),
                         marker='o');

In [ ]:
#Рассчитаем коэффициент корреляции
data_pivot_spb_mean['city_centers_nearest_km'].corr(data_pivot_spb_mean['center_km_price'])

In [ ]:
#Посторим диаграмму размаха
data_pivot_spb_mean.boxplot('center_km_price');

По данным представленным на диаграммах можно сказать, что вид данной зависимости обратный, так как на графике можно рассмотреть убывающую гиперболу. На диаграмме видно, что при приближении к нулю по оси Х, цена для большинства квартир возрастает в несколько раз. То есть наблюдается обратно пропорциональная зависимость. Коэффициент корреляции показывает умеренную зависимость. 
Сренее и медианное значения имеют небольшое отклонение друг от друга в масштабах выборки.

Можно сделать вывод, что удаленность от центра является одним из значительных факторов, в совокупности с другими, влияющими на стоимость квартиры.

### Общий вывод

В результате проведенного анализа, были выполнеы следующие этапы:
- предобработка данных (были выявлены дубликаты, удалены и заполнены пустые значения, выявлены и исправлены аномалии)
- на основе обработанных данных были добавлены необходимые для дальнейшего анализа графы
- был проведен анализ как данных по отдельности, так и в совокупности

По отдельности на гистограммах можно было проследить, что все количественные параметры подчиняются закону растределения Пуассона.

По результатам анализа отношений различных факторов к цене квартиры можно утверждать следующее: 

1. цена прямо пропорциональна площади квартиры, также как площади отдельных частей квартиры

Коэффициент коррекляции составляет 74% для зависимости цены квартиры от общей площади, что свидетельствует о достаточно сильной линейной зависимости.

2. цена прямо пропорциональна количеству комнат

Коэффициент корреляции составил 95%, что свидетельствует о практически абсолютной линейной зависимости.

3. цена напрямую зависит от этажа, на котором находится

Первые и последние этажи чаще всего имеют стоимоть ниже, чем квартиры на других этажах. Возможными причинами могут быть бытовые особенности.

4. цена на квартиру указывается выше в будние дни

Согласно диаграмме стоимость квартиры выставлялась выше в рабочие дни, для выходных, стоимость квартир граничит с нижним порогом. Возможной причиной может быть использование ресурса агентами по недвижимости.

5.  цена слабо взаимосвязана с месяцем, в который было размещено объявление 

Общая стоимость квартиры слабо зависит от месяца регистрации объявления. Стоимость в данном случае зависит от сторонних факторов.

6. цена на квартиры с 2017 года начла расти

Стоимость квартир с 2014 года по 2017 уменьшалась, в 2017 году замечено восходящее движение графика. Возможной причиной могут быть сторонние факторы, такие как инфляция и ситуация в стране.

7. цена обратно пропорциональна расстоянию до центра населенного пункта

Ближе к центру наблюдаются аномально высокие цены на недвижимость. Коэффициент корреляции составил -60%. Возможные причины близость культурного центра, близость располоения необходимых инстанций.

Из полученых результатов можно заключить, что на формирование цены на рынке недвижимости влияют различные факторы в совокупности, начиная от количественных  характеристик до картографических данных. Для выявления более точных возможных факторов ценообразования необходимо проводить точечный анализ требуемых величин.